## Architecture to Monitor Data Quality Over Time

**Description**: Design a monitoring system in Python that checks and logs data quality metrics (accuracy, completeness) for a dataset over time.

**Steps to follow:**
1. Implement a Scheduled Script:
    - Use schedule library to periodically run a script.
2. Script to Calculate Metrics:
    - For simplicity, use a function calculate_quality_metrics() that calculates and logs metrics such as missing rate or mismatch rate.
3. Store Logs:
    - Use Python's logging library to save these metrics over time.

In [ ]:
# Write your code from here
import pandas as pd
import schedule
import time
import logging
from datetime import datetime

# Configure logging
logging.basicConfig(
    filename="data_quality_logs.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# Define path to your dataset
DATA_PATH = "your_dataset.csv"  # Update with your dataset path

# Define trusted dataset for accuracy comparison if needed
TRUSTED_DATA_PATH = "trusted_dataset.csv"  # Optional for accuracy check

# Function to calculate quality metrics
def calculate_quality_metrics():
    try:
        df = pd.read_csv(DATA_PATH)

        # Completeness: % of missing values
        completeness = 100 - df.isnull().mean().mean() * 100

        # Accuracy (optional): if a trusted source exists
        try:
            trusted_df = pd.read_csv(TRUSTED_DATA_PATH)
            # Assume we compare a common field, e.g., 'price'
            merged = df.merge(trusted_df, on='id', suffixes=('', '_trusted'))
            accuracy = (merged['price'] == merged['price_trusted']).mean() * 100
        except Exception:
            accuracy = "Not calculated (trusted data unavailable)"

        # Log metrics
        logging.info(f"Completeness: {completeness:.2f}% | Accuracy: {accuracy}")
        print(f"[{datetime.now()}] Completeness: {completeness:.2f}% | Accuracy: {accuracy}")

    except Exception as e:
        logging.error(f"Error during data quality check: {str(e)}")

# 3. Schedule the task every hour (or daily)
schedule.every(1).hours.do(calculate_quality_metrics)

# 4. Run the schedule continuously
print("Monitoring started... Press Ctrl+C to stop.")
while True:
    schedule.run_pending()
    time.sleep(1)
